In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, get_feature_names

In [2]:
data = pd.read_csv("/Users/yangkaixiang/Desktop/movielens_sample.txt")
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [3]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [6]:
# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=100, verbose=True, validation_split=0.2, )

Epoch 1/100


/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


1/1 [==============================] - 0s 366ms/step - loss: 14.3984 - mse: 14.3984 - val_loss: 13.4909 - val_mse: 13.4909
Epoch 2/100
1/1 [==============================] - 0s 36ms/step - loss: 14.2704 - mse: 14.2704 - val_loss: 13.3834 - val_mse: 13.3834
Epoch 3/100
1/1 [==============================] - 0s 42ms/step - loss: 14.1343 - mse: 14.1343 - val_loss: 13.2684 - val_mse: 13.2684
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 13.9890 - mse: 13.9890 - val_loss: 13.1461 - val_mse: 13.1461
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 13.8347 - mse: 13.8347 - val_loss: 13.0158 - val_mse: 13.0158
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 13.6707 - mse: 13.6707 - val_loss: 12.8776 - val_mse: 12.8776
Epoch 7/100
1/1 [==============================] - 0s 35ms/step - loss: 13.4965 - mse: 13.4965 - val_loss: 12.7308 - val_mse: 12.7308
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1

In [7]:
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

test RMSE 1.5400973995173164
